[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ArtIC-TITECH/b3-proj-2024/blob/main/docs/theme_A.ipynb)

In [1]:
!pip install -U kaleido
import kaleido

import os
import numpy as np
import torch
import plotly.graph_objects as go
import imageio

# 画像保存用のディレクトリを作成
output_dir = "frames"
os.makedirs(output_dir, exist_ok=True)

# 2次元の関数 (例: f(x, y) = x^2 - 0.1 * y^2)
def func(x, y):
    return x**2 - 0.1*y**2

# 勾配 (微分)
def grad_func(x, y):
    dx = 2 * x
    dy = -0.2 * y
    return torch.tensor([dx, dy])

# 初期値設定
x_init = torch.tensor([9.0, 1.0], requires_grad=True)
lr = 0.9  # 学習率
epochs = 15  # イテレーション数

# 最適化過程を保存
history = []

# SGD最適化
for epoch in range(epochs):
    grad = grad_func(x_init[0], x_init[1])
    with torch.no_grad():
        x_init -= lr * grad
    history.append(x_init.clone().detach().numpy())

# 3Dグラフの準備
x = np.linspace(-10, 10, 100)
y = np.linspace(0, 16, 100)
X, Y = np.meshgrid(x, y)
Z = func(X, Y)

# 最適化の軌跡
history = np.array(history)
trajectory = func(history[:, 0], history[:, 1])

# plotlyで3Dプロットを作成
fig = go.Figure()

# 関数の3Dサーフェス
fig.add_trace(go.Surface(z=Z, x=X, y=Y, colorscale='Viridis', opacity=0.6))

# 最適化の軌跡
fig.add_trace(go.Scatter3d(
    x=history[:, 0], y=history[:, 1], z=trajectory,
    mode='markers+lines',
    marker=dict(size=5, color='red', opacity=1),
    line=dict(color='red', width=2)
))

# タイトルとラベル
fig.update_layout(
    title="Optimization Process without Momentum",
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='f(x, y)'
    ),
    width=800,
    height=600
)

# GIFの生成
frames = []
for angle in range(0, 360, 10):  # 10度ずつ回転
    fig.update_layout(scene_camera_eye=dict(x=1.25 * np.sin(np.radians(angle)), y=1.25 * np.cos(np.radians(angle)), z=0.5))

    # 画像を専用フォルダに保存
    filename = os.path.join(output_dir, f"frame_{angle}.png")
    fig.write_image(filename)

    # フレームを読み込んでリストに追加
    frames.append(imageio.imread(filename))

# GIF保存
imageio.mimsave("without_momentum.gif", frames, fps=10)


<ipython-input-1-2d9c67b030be>:85: DeprecationWarning:

Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.



In [2]:
# 画像保存用のディレクトリを作成
output_dir = "frames"
os.makedirs(output_dir, exist_ok=True)

# 2次元の関数 (例: f(x, y) = x^2 - 0.1 * y^2)
def func(x, y):
    return x**2 - 0.1*y**2

# 勾配 (微分)
def grad_func(x, y):
    dx = 2 * x
    dy = -0.2 * y
    return torch.tensor([dx, dy])

# 初期値設定
x_init = torch.tensor([9.0, 1.0], requires_grad=True)
lr = 0.9  # 学習率
epochs = 15  # イテレーション数

# 初期値設定
x_init = torch.tensor([9.0, 1.0], requires_grad=True)
lr = 0.9  # 学習率
epochs = 15  # イテレーション数

# 最適化過程を保存
history = []

# モーメンタム法による最適化
momentum = 0.1  # モーメンタムのパラメータ
v = torch.zeros_like(x_init)  # 更新量

for epoch in range(epochs):
    grad = grad_func(x_init[0], x_init[1])

    # モーメンタムの更新
    v = momentum * v + grad
    with torch.no_grad():
        x_init -= lr * v  # 更新量に学習率を掛けて更新

    # 最適化の進行状況を保存
    history.append(x_init.clone().detach().numpy())

# 3Dグラフの準備
x = np.linspace(-10, 10, 100)
y = np.linspace(0, 16, 100)
X, Y = np.meshgrid(x, y)
Z = func(X, Y)

# 最適化の軌跡
history = np.array(history)
trajectory = func(history[:, 0], history[:, 1])

# plotlyで3Dプロットを作成
fig = go.Figure()

# 関数の3Dサーフェス
fig.add_trace(go.Surface(z=Z, x=X, y=Y, colorscale='Viridis', opacity=0.6))

# 最適化の軌跡
fig.add_trace(go.Scatter3d(
    x=history[:, 0], y=history[:, 1], z=trajectory,
    mode='markers+lines',
    marker=dict(size=5, color='red', opacity=1),
    line=dict(color='red', width=2)
))

# タイトルとラベル
fig.update_layout(
    title="Optimization Process with Momentum",
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='f(x, y)'
    ),
    width=800,
    height=600
)

# GIFの生成
frames = []
for angle in range(0, 360, 10):  # 10度ずつ回転
    fig.update_layout(scene_camera_eye=dict(x=1.25 * np.sin(np.radians(angle)), y=1.25 * np.cos(np.radians(angle)), z=0.5))

    # 画像を専用フォルダに保存
    filename = os.path.join(output_dir, f"frame_{angle}.png")
    fig.write_image(filename)

    # フレームを読み込んでリストに追加
    frames.append(imageio.imread(filename))

# GIF保存
imageio.mimsave("with_momentum.gif", frames, fps=10)


<ipython-input-2-91f77a1be3d5>:89: DeprecationWarning:

Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.

